In [33]:
import spectral
import matplotlib.pyplot as plt
import numpy as np
import spectral.io.aviris as aviris
import pandas as pd
import pysptools.eea as eea
import pysptools.noise as noise
import pandas as pd
import os

In [12]:
img = aviris.open('C:/Users/gabri/RFL0/f970619t01p02_r02_sc02.a.rfl', 'C:/Users/gabri/f970619t01p02_r02.a.spc')

In [13]:
img

	Data Source:   'C:/Users/gabri/RFL0/f970619t01p02_r02_sc02.a.rfl'
	# Rows:            512
	# Samples:         614
	# Bands:           224
	Interleave:        BIP
	Quantization:  16 bits
	Data format:     int16

# PURE PIXEL EXTRACTION

In [14]:
img_data=img.load()
    
np.set_printoptions(threshold=np.inf, linewidth=140)
    
img_2d=img_data.reshape((img_data.shape[0]*img_data.shape[1]), img_data.shape[2])

In [5]:
[purepixels,bands]=eea.eea.PPI(img_2d,10,10)

In [15]:
bands

array([111958, 104790, 104783, 104784, 104785, 104786, 104787, 104788, 104789, 104791], dtype=int64)

# WAVELENGTH EXTRACTION OF PPI

In [16]:
file_path = 'C:/Users/gabri/f970619t01p02_r02.a.spc'

In [17]:
column_names =['wavelength_center_position','fwhm_equivalent_gaussian','Uncertainty in wavelength center position',                             
                           'Uncertainty in FWHM for equivalent gaussian',                           
                           'channel_number']

In [18]:
df_wav_ppi = pd.read_csv(file_path, delimiter='\t', header=None, names=column_names)

In [19]:
wavelength=[]
wavelength = df_wav_ppi.wavelength_center_position 

In [20]:
len(wavelength)

224

In [21]:
wavelength[:10]

0    369.85
1    379.69
2    389.53
3    399.37
4    409.21
5    419.06
6    428.91
7    438.76
8    448.61
9    458.46
Name: wavelength_center_position, dtype: float64

In [22]:
max_ppi,min_ppi=max(wavelength),min(wavelength)

In [23]:
print(max_ppi)
print(min_ppi)

2506.81
369.85


In [31]:
ppi_wavelen=[]
for wave in wavelength:
    we=float(round(wave))
    ppi_wavelen.append(we)
wave_spec= pd.DataFrame({'wavelength':ppi_wavelen})

# PPI DATAFRAME

In [17]:
bands

array([111958, 104790, 104783, 104784, 104785, 104786, 104787, 104788, 104789, 104791], dtype=int64)

In [27]:
def read_and_process_file(file_path, wave_spec):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # Check if the line is not empty
                if "Name: " in line:
                    name_value = line.split("Name: ")[1].strip()
                elif "Class: " in line:
                    class_value = line.split("Class: ")[1].strip()
                elif len(line.split()) == 2 and all(value.replace('.', '').isdigit() for value in line.split()):
                    # Check if the line has two elements and both are numeric
                    data.append([float(value) for value in line.split()])

    df = pd.DataFrame(data, columns=['wavelength', f'reflectance#{class_value}#{name_value}'])
    df['wavelength'] = pd.to_numeric(df['wavelength'], errors='coerce') * 1000
    df[f'reflectance#{class_value}#{name_value}'] = pd.to_numeric(df[f'reflectance#{class_value}#{name_value}'], errors='coerce')
    return df[df['wavelength'].isin(wave_spec['wavelength']) == True]


In [29]:
def merge_files_into_dataframe(directory, wave_spec):
    dfs = pd.DataFrame({'wavelength': wave_spec['wavelength']})
    
    for filename in os.listdir(directory):
        if filename.endswith('.spectrum.txt'):
            file_path = os.path.join(directory, filename)
            df = read_and_process_file(file_path, wave_spec)
            dfs = pd.merge(dfs, df, on='wavelength', how='left')
    
    return dfs

In [34]:
directory = 'D:/data-20231104T081748Z-001/beckman/'
result_dataframe = merge_files_into_dataframe(directory, wave_spec)
print(result_dataframe)

C:\Users\gabri\AppData\Local\Temp\ipykernel_16612\3938279286.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'reflectance#Carbonates#Calcite CaCO_3_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfs = pd.merge(dfs, df, on='wavelength', how='left')
C:\Users\gabri\AppData\Local\Temp\ipykernel_16612\3938279286.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'reflectance#Carbonates#Calcite CaCO_3_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfs = pd.merge(dfs, df, on='wavelength', how='left')
C:\Users\gabri\AppData\Local\Temp\ipykernel_16612\3938279286.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'reflectance#Carbonates#Dolomite CaMg(CO_3)_2_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfs = pd.merge(dfs, df, on='wavelength', how='left')
C:\Users\gabri\AppData\Local\Temp\ipykernel_16612\3938279286.py:8: Futur

     wavelength  reflectance#Arsenates#Mimetite Pb_5(AsO_4)_3Cl_x  \
0         370.0                                               NaN   
1         380.0                                               NaN   
2         390.0                                               NaN   
3         399.0                                               NaN   
4         409.0                                             13.05   
..          ...                                               ...   
219      2467.0                                               NaN   
220      2477.0                                               NaN   
221      2487.0                                               NaN   
222      2497.0                                               NaN   
223      2507.0                                               NaN   

     reflectance#Arsenates#Scorodite Fe^3+AsO_4.2H_2O_x  \
0                                                  NaN    
1                                                  Na

In [35]:
df_transposed = result_dataframe.transpose().reset_index()
df_transposed.set_axis(df_transposed.iloc[0], axis=1, inplace=True)
df_transposed = df_transposed[1:]
df_transposed[['value', 'class', 'sub - class']] = df_transposed['wavelength'].str.split("#",expand=True)

columns_to_drop = ['value', 'wavelength']
df_transposed = df_transposed.drop(columns=columns_to_drop)

filtered_columns = df_transposed.columns[df_transposed.notnull().all()]
filtered_df = df_transposed[filtered_columns]

col=list(filtered_df.columns)[:-2]

filtered_df['class'].value_counts()

C:\Users\gabri\AppData\Local\Temp\ipykernel_16612\1637771729.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df_transposed.set_axis(df_transposed.iloc[0], axis=1, inplace=True)


Silicates     216
Carbonates     45
Sulfates       39
Sulfides       34
Oxides         31
Borates        18
Halides        15
Phosphates     12
Arsenates       6
Elements        6
Hydroxides      5
Tungstates      3
Name: class, dtype: int64

In [38]:
filtered_df.head()

,409.0,419.0,429.0,439.0,449.0,458.0,468.0,478.0,488.0,498.0,...,2268.0,2288.0,2308.0,2328.0,2348.0,2368.0,2388.0,2408.0,class,sub - class
1,13.05,13.62,14.30,15.26,16.67,18.35,20.85,23.93,27.55,31.83,...,66.23,66.54,66.13,65.83,66.28,66.43,66.15,65.96,Arsenates,Mimetite Pb_5(AsO_4)_3Cl_x
2,7.78,7.58,7.22,8.47,10.38,11.01,11.53,12.57,14.02,15.78,...,8.89,8.56,7.54,7.07,7.22,7.06,6.63,6.11,Arsenates,Scorodite Fe^3+AsO_4.2H_2O_x
3,60.13,61.51,63.37,65.49,67.67,69.63,72.14,74.54,76.63,78.82,...,91.49,91.59,91.53,91.73,91.81,91.84,91.69,91.65,Arsenates,Mimetite Pb_5(AsO_4)_3Cl_y
4,29.67,30.59,30.79,36.28,41.59,43.15,44.75,47.49,51.50,54.92,...,55.51,54.90,53.02,52.09,52.37,51.98,50.92,49.54,Arsenates,Scorodite Fe^3+AsO_4.2H_2O_y
5,27.81,29.23,31.25,33.68,36.52,39.37,42.97,47.06,51.34,55.70,...,80.62,80.80,80.45,80.51,80.91,80.94,80.83,80.70,Arsenates,Mimetite Pb_5(AsO_4)_3Cl


In [36]:
row_indices_to_print = list(bands) # list of ppi bands

specific_rows = img_2d[row_indices_to_print,:]

df_pppi = pd.DataFrame(specific_rows.T, columns=row_indices_to_print)

df_pppi.insert(0, 'wavelength', ppi_wavelen) #inserting the purepixel wavelengths

selected_df2 = df_pppi[df_pppi['wavelength'].isin(col)]

selecteddf_t=selected_df2.T
selecteddf_t.set_axis(selecteddf_t.iloc[0], axis=1, inplace=True)
selecteddf_t= selecteddf_t[1:]

selecteddf_t

C:\Users\gabri\AppData\Local\Temp\ipykernel_16612\2060080328.py:12: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  selecteddf_t.set_axis(selecteddf_t.iloc[0], axis=1, inplace=True)


wavelength,409.0,419.0,429.0,439.0,449.0,458.0,468.0,478.0,488.0,498.0,...,1928.0,2248.0,2268.0,2288.0,2308.0,2328.0,2348.0,2368.0,2388.0,2408.0
111958,0.2846,0.3114,0.3287,0.3425,0.3538,0.3585,0.3612,0.3657,0.3714,0.3800,...,0.0,0.5891,0.5879,0.5866,0.5802,0.5858,0.5857,0.5723,0.5696,0.5482
104790,0.1111,0.1195,0.1285,0.1331,0.1378,0.1430,0.1459,0.1479,0.1511,0.1547,...,0.0,0.2904,0.2893,0.2855,0.2763,0.2644,0.2611,0.2608,0.2608,0.2410
104783,0.1111,0.1218,0.1337,0.1380,0.1483,0.1521,0.1535,0.1562,0.1595,0.1638,...,0.0,0.2510,0.2534,0.2539,0.2460,0.2399,0.2372,0.2341,0.2297,0.2189
104784,0.1111,0.1231,0.1327,0.1422,0.1502,0.1543,0.1571,0.1603,0.1636,0.1689,...,0.0,0.2566,0.2524,0.2568,0.2447,0.2398,0.2313,0.2302,0.2251,0.2206
104785,0.1127,0.1231,0.1327,0.1396,0.1502,0.1543,0.1556,0.1590,0.1636,0.1689,...,0.0,0.2702,0.2648,0.2655,0.2560,0.2470,0.2405,0.2420,0.2387,0.2288
104786,0.1111,0.1231,0.1327,0.1422,0.1496,0.1532,0.1550,0.1575,0.1622,0.1664,...,0.0,0.2662,0.2698,0.2667,0.2625,0.2529,0.2405,0.2403,0.2439,0.2268
104787,0.1046,0.1148,0.1244,0.1305,0.1391,0.1441,0.1485,0.1501,0.1538,0.1580,...,0.0,0.2596,0.2583,0.2527,0.2460,0.2413,0.2375,0.2343,0.2352,0.2295
104788,0.0966,0.1053,0.1119,0.1216,0.1299,0.1340,0.1372,0.1395,0.1435,0.1480,...,0.0,0.2778,0.2757,0.2744,0.2650,0.2615,0.2525,0.2551,0.2511,0.2358
104789,0.0966,0.1018,0.1098,0.1165,0.1214,0.1267,0.1276,0.1317,0.1343,0.1388,...,0.0,0.2930,0.2871,0.2814,0.2663,0.2644,0.2612,0.2556,0.2456,0.2333
104791,0.1225,0.1302,0.1410,0.1488,0.1554,0.1605,0.1632,0.1650,0.1689,0.1735,...,0.0,0.2930,0.2921,0.2901,0.2817,0.2682,0.2668,0.2624,0.2585,0.2484


# SPECTRAL LIBRARY DATAFRAME

In [49]:
def spectral_angle_mapper(spectrum1,spectrum2):
    
    num = np.dot(spectrum1, spectrum2)

        # Calculate the sum of squares separately for each array
    sum_of_squares_array1 = np.sum(spectrum1 ** 2)
    sum_of_squares_array2 = np.sum(spectrum2 ** 2)

    # Take the product of the sums of squares
    product_of_sums_of_squares = sum_of_squares_array1 * sum_of_squares_array2

    # Calculate the square root of the product
    denom = np.sqrt(product_of_sums_of_squares)
    arccosine_value = np.arccos(num/denom)

     # Assign the result to the variable 'sam'
    sam = arccosine_value

    return sam

In [50]:
sam = spectral_angle_mapper(filtered_df.iloc[1, :85].values,selecteddf_t.iloc[1,:].values )

In [51]:
sam

0.5497262322338511

In [57]:
import pandas as pd

# Assuming you have a function spectral_angle_mapper defined

# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

# Assuming selecteddf_t and filtered_df are DataFrames
for index1, row1 in selecteddf_t.iterrows():
   
    for index2, row2 in filtered_df.iterrows():
        
        

print(result_df)


TypeError: list expected at most 1 argument, got 2

In [64]:
# Get the number of rows in df1 and df2
num_rows_df1 = len(selecteddf_t)
num_rows_df2 = len(filtered_df)

# Create a new dataframe with the specified dimensions
new_dataframe = pd.DataFrame(index=range(num_rows_df2))



In [65]:
new_dataframe.head()

""
0
1
2
3
4


In [66]:
for index1, row1 in selecteddf_t.iterrows():
   
    for index2, row2 in filtered_df.iterrows():
        
            sam = spectral_angle_mapper(row2[:85].values, row1.values)

            new_dataframe.at[index2, index1] = sam

In [71]:
new_dataframe=new_dataframe[1:]

In [72]:
new_dataframe.shape

(430, 11)

In [73]:
new_dataframe['mineral'] = filtered_df['class'].astype(str) + " _ " + filtered_df['sub - class'].astype(str)

In [82]:
result= new_dataframe.T
result.columns = result.loc['mineral']
result = result.drop('mineral')


In [83]:
result.head()

mineral,Arsenates _ Mimetite Pb_5(AsO_4)_3Cl_x,Arsenates _ Scorodite Fe^3+AsO_4.2H_2O_x,Arsenates _ Mimetite Pb_5(AsO_4)_3Cl_y,Arsenates _ Scorodite Fe^3+AsO_4.2H_2O_y,Arsenates _ Mimetite Pb_5(AsO_4)_3Cl,Arsenates _ Scorodite Fe^3+AsO_4.2H_2O,Borates _ Colemanite Ca_2B_6O_11.5H_2O_x,Borates _ Kernite Na_2B_4O_6(OH)_2.3H_2O_x,Borates _ Ulexite NaCaB_5O_6(OH)_6.5H_2O_x,Borates _ Tincalconite Na_2B_4O_5(OH)_4.3H_2O_x,...,Sulfides _ Arsenopyrite FeAsS,Sulfides _ Stibnite Sb_2S_3,Sulfides _ Galena PbS,Sulfides _ Chalcocite Cu_2S,Sulfides _ Bornite Cu_5FeS_4,Sulfides _ Marcasite FeS_2,Sulfides _ Molybdenite MoS_2,Tungstates _ Scheelite CaWO_4_x,Tungstates _ Scheelite CaWO_4_y,Tungstates _ Scheelite CaWO_4
111958,0.393368,0.523225,0.371078,0.367691,0.373407,0.454939,0.55547,0.443645,0.596515,0.497668,...,0.383064,0.668152,0.417462,0.406916,0.488338,0.643037,0.561099,0.385565,0.375447,0.388986
104790,0.405078,0.549726,0.40665,0.402456,0.397521,0.479354,0.615032,0.499617,0.657348,0.556863,...,0.421513,0.645061,0.468725,0.459769,0.488883,0.622563,0.553427,0.403081,0.406907,0.408093
104783,0.387933,0.516992,0.374651,0.363169,0.371248,0.446736,0.537717,0.433924,0.577393,0.481961,...,0.390038,0.691999,0.42379,0.414109,0.510952,0.664832,0.587018,0.392285,0.381423,0.396455
104784,0.391208,0.521521,0.377708,0.366773,0.374514,0.451049,0.542167,0.437355,0.58178,0.48592,...,0.392837,0.691706,0.426762,0.417243,0.511988,0.665235,0.587321,0.394107,0.384225,0.398483
104785,0.502063,0.569718,0.508818,0.486998,0.499558,0.519038,0.678786,0.567021,0.717954,0.623669,...,0.522411,0.713003,0.558367,0.549697,0.581958,0.702888,0.641124,0.512461,0.510527,0.514282


In [88]:
result_numeric = result.apply(pd.to_numeric, errors='coerce')

# Create a new column with the index of the minimum value in each row
result['classification'] = result_numeric.idxmin(axis=1)

# Create another column with the actual minimum values
result['min_value'] = result_numeric.min(axis=1)

In [89]:
result

mineral,Arsenates _ Mimetite Pb_5(AsO_4)_3Cl_x,Arsenates _ Scorodite Fe^3+AsO_4.2H_2O_x,Arsenates _ Mimetite Pb_5(AsO_4)_3Cl_y,Arsenates _ Scorodite Fe^3+AsO_4.2H_2O_y,Arsenates _ Mimetite Pb_5(AsO_4)_3Cl,Arsenates _ Scorodite Fe^3+AsO_4.2H_2O,Borates _ Colemanite Ca_2B_6O_11.5H_2O_x,Borates _ Kernite Na_2B_4O_6(OH)_2.3H_2O_x,Borates _ Ulexite NaCaB_5O_6(OH)_6.5H_2O_x,Borates _ Tincalconite Na_2B_4O_5(OH)_4.3H_2O_x,...,Sulfides _ Galena PbS,Sulfides _ Chalcocite Cu_2S,Sulfides _ Bornite Cu_5FeS_4,Sulfides _ Marcasite FeS_2,Sulfides _ Molybdenite MoS_2,Tungstates _ Scheelite CaWO_4_x,Tungstates _ Scheelite CaWO_4_y,Tungstates _ Scheelite CaWO_4,min_value,classification
111958,0.393368,0.523225,0.371078,0.367691,0.373407,0.454939,0.55547,0.443645,0.596515,0.497668,...,0.417462,0.406916,0.488338,0.643037,0.561099,0.385565,0.375447,0.388986,0.344234,Silicates _ Analcime NaAlSi_2O_6.H_2O
104790,0.405078,0.549726,0.40665,0.402456,0.397521,0.479354,0.615032,0.499617,0.657348,0.556863,...,0.468725,0.459769,0.488883,0.622563,0.553427,0.403081,0.406907,0.408093,0.381197,"Silicates _ Nontronite Na_0.33Fe_2^3+(Si,Al)_4..."
104783,0.387933,0.516992,0.374651,0.363169,0.371248,0.446736,0.537717,0.433924,0.577393,0.481961,...,0.42379,0.414109,0.510952,0.664832,0.587018,0.392285,0.381423,0.396455,0.340070,"Silicates _ Saponite (Ca/2,Na)_0.33(Mg,Fe^2+)_..."
104784,0.391208,0.521521,0.377708,0.366773,0.374514,0.451049,0.542167,0.437355,0.58178,0.48592,...,0.426762,0.417243,0.511988,0.665235,0.587321,0.394107,0.384225,0.398483,0.342496,"Silicates _ Saponite (Ca/2,Na)_0.33(Mg,Fe^2+)_..."
104785,0.502063,0.569718,0.508818,0.486998,0.499558,0.519038,0.678786,0.567021,0.717954,0.623669,...,0.558367,0.549697,0.581958,0.702888,0.641124,0.512461,0.510527,0.514282,0.486998,Arsenates _ Scorodite Fe^3+AsO_4.2H_2O_y
104786,0.504962,0.571313,0.510959,0.489525,0.502126,0.521252,0.680006,0.568759,0.718821,0.62523,...,0.559743,0.551075,0.583698,0.704267,0.642636,0.51515,0.512679,0.516769,0.489525,Arsenates _ Scorodite Fe^3+AsO_4.2H_2O_y
104787,0.478808,0.555541,0.487872,0.466278,0.47713,0.501475,0.669486,0.553683,0.709984,0.612651,...,0.541921,0.532894,0.559805,0.682437,0.620119,0.489583,0.489115,0.491729,0.466278,Arsenates _ Scorodite Fe^3+AsO_4.2H_2O_y
104788,0.395088,0.537392,0.402653,0.393008,0.389618,0.466145,0.602214,0.491044,0.644283,0.545413,...,0.467912,0.458658,0.497665,0.631461,0.564409,0.401985,0.40449,0.407153,0.375305,"Silicates _ Nontronite Na_0.33Fe_2^3+(Si,Al)_4..."
104789,0.513779,0.592612,0.537978,0.518166,0.52115,0.54141,0.743976,0.623283,0.786585,0.68658,...,0.60172,0.59333,0.568657,0.666829,0.617412,0.525191,0.535148,0.527928,0.508194,"Silicates _ Nontronite Na_0.33Fe_2^3+(Si,Al)_4..."
104791,0.493615,0.570162,0.501507,0.482718,0.491994,0.517596,0.689258,0.571258,0.729985,0.631981,...,0.554595,0.545824,0.561441,0.679733,0.618372,0.500299,0.501534,0.502322,0.482718,Arsenates _ Scorodite Fe^3+AsO_4.2H_2O_y
